In [40]:
# Building an initial decision tree and seeing how it makes predictions

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] #only looking at petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

#Visualising the trained Decision tree with export_graphviz()

from sklearn.tree import export_graphviz

def image_path(filename):
    return f"/Users/harryfyjis-walker/Desktop/Practice ML techniques/{filename}"

export_graphviz(
        tree_clf,
        out_file=image_path("iris_tree.dot"),
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
)

#To convert to png etc., use command line : dot -Tpng iris_tree.dot -o iris_tree.png

#Making predictions:
#Want to classify an iris flower. Start at root note (depth 0, petal length smaller than 2.45 cm?), if Y move to left child note (depth 1), predicts class=setosa (leaf node). 
#If N, move to right node, "is petal width smaller than 1.75cm?" - if Y, class=veriscolor; if N, class=virginica.
# Gini score of the ith note denotes purity - pi,k is ratio of class k instances among the training  instances in the ith node

In [48]:
#Estimating Class Probabilities
#Decision Trees can also estimate probability that an instance belongs to a particular class k, by ratio of training instances of class k in a leaf node.
tree_clf.predict_proba([[5, 1.5]]) #For an example of a flower whose petals are 5cm long and 1.5cm wide, predict probability of it being setosa, veriscolor, or virginica

array([[0.        , 0.90740741, 0.09259259]])

In [46]:
tree_clf.predict([[5, 1.5]]) #Predict the class for these dimensions - veriscolor

array([1])

In [54]:
#Scikit-Learn uses CART to train decision trees - splits training set into two subsets using a single feature k and a threshold tk (e.g. petal length < 2.45cm). Chooses k and tk by searching for the pair that produces purest subsets (weighted by size):
#Cost function = J(k,tk) = m(left)/m * G(left) + m(right)/m * G(right), where
# G(l/r) measures impurity of l/r subset; m(l/r) measures number of instances (weight)
#Once CART splits training set in two, splits subsets with same logic, and continues recursively until reaching the max_depth (or min_samples split, min_samples_leaf, min_weight_fraction_leaf, and max_leaf_nodes)

#NB entropy can be used as a hyperparameter rather than Gini impurity - entropy = 0 when only one class
#Gini impurity is usually slightly faster, however

In [56]:
#Regularisation:

#Decision trees often overfit because they are "nonparametric" i.e. the number of parameters is not determined prior to training (unlike a linear model, which has a predetermined number of parameters). 
#To avoid overfitting, restrict DT's freedom - usually through max_deptth, which regularises the model, reducing risk of overfitting

In [58]:
#Regression:

#DTs are also capable of performing regression tasks e.g. using DecisionTreeRegressor:

from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

# This predicts a value instead of a class. 
# Instead of trying to split training set to minimise impurity, it tries to minimise MSE
#J(k, tk) = m(l)/m * MSE(l) + m(r)/m * MSE(r); see p.214 for definitions of MSE(node)

DecisionTreeRegressor(max_depth=2)

In [60]:
#Limitations:

#1. Note that decision trees prefer orthogonal decision boundaries (splits perp. to an axis), so are sensitive to training set rotation (can limit this problem with Principal Component Analysis, see later)
#2. Very sensitive to small variations in the training data. Can limit with averaging predictions over many trees, through Random Forests.

In [ ]:
#End-of-chapter Questions:

#1. What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with one million instances?
#The depth of a well-balanced binary tree containing m leaves is equal to log2(m),2 rounded up. A binary Decision Tree (one that makes only binary decisions, as is
#the case with all trees in Scikit-Learn) will end up more or less well balanced at the end of training, with one leaf per training instance if it is trained without
#restrictions. Thus, if the training set contains one million instances, the Decision Tree will have a depth of log2(106) ≈ 20 (actually a bit more since the tree will
#generally not be perfectly well balanced).


#2. Is a node’s Gini impurity generally lower or greater than its parent’s? Is it generally lower/greater, or always lower/greater?
#Generally lower, presuming purification outweighs reduction in number of instances. But this does not always outweigh it.

#A node’s Gini impurity is generally lower than its parent’s. This is due to the CART training algorithm’s cost function, which splits each node in a way that
#minimizes the weighted sum of its children’s Gini impurities. However, it is possi‐ ble for a node to have a higher Gini impurity than its parent, as long as this
#increase is more than compensated for by a decrease in the other child’s impurity. For example, consider a node containing four instances of class A and one of
#class B. Its Gini impurity is 1 – (1/5)2 – (4/5)2 = 0.32. Now suppose the dataset is one-dimensional and the instances are lined up in the following order: A, B, A,
#A, A. You can verify that the algorithm will split this node after the second instance, producing one child node with instances A, B, and the other child node
#with instances A, A, A. The first child node’s Gini impurity is 1 – (1/2)2 – (1/2)2 = 0.5, which is higher than its parent’s. This is compensated for by the fact that the
#other node is pure, so its overall weighted Gini impurity is 2/5 × 0.5 + 3/5 × 0 = 0.2, which is lower than the parent’s Gini impurity.

#3. If a Decision Tree is overfitting the training set, is it a good idea to try decreasing max_depth?
#Yes If a Decision Tree is overfitting the training set, it may be a good idea to decrease
# max_depth, since this will constrain the model, regularizing it.

#4. If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features?

#Decision Trees don’t care whether or not the training data is scaled or centered; that’s one of the nice things about them. So if a Decision Tree underfits the train‐ing set, scaling the input features will just be a waste of time.

#5. If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances?

#The computational complexity of training a Decision Tree is O(n × m log(m)). So if you multiply the training set size by 10, the training time will be multiplied by
#K = (n × 10m × log(10m)) / (n × m × log(m)) = 10 × log(10m) / log(m). If m = 106, then K ≈ 11.7, so you can expect the training time to be roughly 11.7 hours.

#6. If your training set contains 100,000 instances, will setting presort=True speed up training?

#Presorting the training set speeds up training only if the dataset is smaller than a few thousand instances. If it contains 100,000 instances, setting presort=True
#will considerably slow down training.

In [64]:
#Q7.  Train and fine-tune a Decision Tree for the moons dataset by following these steps:
# a. Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.
# Use train_test_split() to split the dataset into a training set and a test set.
#c. Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier.
#Hint: try various values for max_leaf_nodes.
# d. Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87%
#accuracy.

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#hyperparameters possibilities: min_samples_split, max_depth, min_samples_leaf, min_weight_fraction_leaf, max_leaf_nodes, max_features
#Gridsearchcv automates the hyperparameter tuning process (trains and evaluates model using different combinations of hyperparameters)
params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)

grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [66]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=17, random_state=42)

In [68]:
#To evaluate the model's accuracy:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

In [70]:
#Q8: a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit- Learn’s ShuffleSplit class for this.
#b. Train one Decision Tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely
#perform worse than the first Decision Tree, achieving only about 80% accuracy.
# c. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This approach gives you majority-vote predictions 
#over the test set. d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


